#Zadanie 7 (7 pkt)
Celem zadania jest zaimplementowanie dwóch wersji naiwnego klasyfikatora Bayesa.
* W pierwszej wersji należy dokonać dyskretyzacji danych - przedział wartości każdego atrybutu dzielimy na cztery równe przedziały i każdej ciągłej wartości atrybutu przypisujemy wartość dyskretną wynikająca z przynależności do danego przedziału.
* W drugiej wersji wartości likelihood wyliczamy z rozkładów normalnych o średnich i odchyleniach standardowych wynikających z wartości atrybutów.
Trening i test należy przeprowadzić dla zbioru Iris, tak jak w przypadku zadania z drzewem klasyfikacyjnym. Proszę przeprowadzić eksperymenty najpierw dla DOKŁADNIE takiego podziału zbioru testowego i treningowego jak umieszczony poniżej. W dalszej części należy przeprowadzić analizę działania klasyfikatorów dla różnych wartości parametrów. Proszę korzystać z przygotowanego szkieletu programu, oczywiście można go modyfikować według potrzeb. Wszelkie elementy szkieletu zostaną wyjaśnione na zajęciach.

* Dyskretyzacja danych - **0.5 pkt**
* Implementacja funkcji rozkładu normalnego o zadanej średniej i odchyleniu standardowym. - **0.5 pkt**
* Implementacja naiwnego klasyfikatora Bayesa dla danych dyskretnych. - **2.0 pkt**
* Implementacja naiwnego klasyfikatora Bayesa dla danych ciągłych. - **2.5 pkt**
* Przeprowadzenie eksperymentów, wnioski i sposób ich prezentacji. - **1.5 pkt**

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import math
from collections import Counter
import numpy as np

iris = load_iris()

x = iris.data
y = iris.target

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=123)

In [ ]:
def data_discretization(data, number_of_discrete_values):
    new_data = np.zeros_like(data, dtype='int32')
    for column_index in range(data.shape[-1]):
        max_column_value = np.max(data[:,column_index])
        min_column_value = np.min(data[:,column_index])
        disrete_range = (max_column_value-min_column_value) * 1/number_of_discrete_values
        discrete_boundaries = np.linspace(min_column_value, max_column_value, number_of_discrete_values+1)[1:-1]
        new_data[:, column_index] = np.digitize(data[:, column_index], discrete_boundaries, right=True).astype('int32')
    return new_data

In [ ]:
x_train_discrete = data_discretization(x_train, 4)
x_test_discrete = data_discretization(x_test, 4)

In [157]:
from collections import Counter
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

iris = load_iris()

x = iris.data
y = iris.target

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=123)


class NaiveBayes:
    def __init__(self):
        self.priors = {}
        self.likelihoods = {}  # discrete for each feature index {0: {0: 0.5, 1: 0.5,...},...}

    def build_classifier(self, train_features, train_classes):
        class_counter = Counter(train_classes)
        self.priors = {key: class_items_number / len(train_classes) for key, class_items_number in class_counter.items()}
        train_features_discrete = self.data_discretization(train_features)

        for column_index in range(train_features_discrete.shape[-1]):
            feature_counter = Counter(train_features_discrete[:,column_index])
            self.likelihoods[column_index] = {key: feature_items_number / len(train_features_discrete) for key, feature_items_number in feature_counter.items()}  #staticmethod

    def data_discretization(self, data, number_of_discrete_values=4):
        new_data = np.zeros_like(data, dtype='int32')
        self.discrete_boundaries = np.zeros([data.shape[-1], number_of_discrete_values-1])
        for column_index in range(data.shape[-1]):
            max_column_value = np.max(data[:,column_index])
            min_column_value = np.min(data[:,column_index])
            disrete_range = (max_column_value-min_column_value) * 1/number_of_discrete_values
            self.discrete_boundaries[column_index] = np.linspace(min_column_value, max_column_value, number_of_discrete_values+1)[1:-1]
            new_data[:, column_index] = np.digitize(data[:, column_index], self.discrete_boundaries[column_index], right=True).astype('int32')
        return new_data

    '''@staticmethod
    def data_discretization_2(data, number_of_discrete_values=4):
        new_data = np.zeros_like(data, dtype='int32')
        for column_index in range(data.shape[-1]):
            max_column_value = np.max(data[:,column_index])
            min_column_value = np.min(data[:,column_index])
            disrete_range = (max_column_value-min_column_value) * 1/number_of_discrete_values
            discrete_boundaries = np.linspace(min_column_value, max_column_value, number_of_discrete_values+1)[1:-1]
            new_data[:, column_index] = np.digitize(data[:, column_index], discrete_boundaries, right=True).astype('int32')
        return new_data
    '''

    def sample_discretization(self, sample):
        return np.array([np.digitize(feature, self.discrete_boundaries[i], right=True).astype('int32') for i,feature in enumerate(sample)])

    def predict(self, sample):
        print(sample)
        discrete_sample = self.sample_discretization(sample)
        likelihoods_list = [self.likelihoods[i][feature] for i, feature in enumerate(discrete_sample)]
        print(discrete_sample)
        print(likelihoods_list)


class GaussianNaiveBayes:
    def __init__(self):
        self.priors = {}
        self.likelihoods = {}  # for each feature index fe. {0: [mean, std],...}

    def build_classifier(self, train_features, train_classes):
        class_counter = Counter(train_classes)
        self.priors = {key: class_item_number / len(train_classes) for key, class_item_number in class_counter.items()}
        for column_index in range(train_features.shape[-1]):
            mean = np.mean(train_features[:,column_index])
            std = np.sqrt(np.mean(np.power(train_features[:,column_index],2)) - np.power(mean, 2))
            self.likelihoods[column_index] = [mean, std]

    @staticmethod
    def normal_dist(x, mean, std):
        return (1/(std*np.sqrt(2*np.pi))) * np.exp((-np.power(x-mean, 2))/(2*np.power(std, 2)))

    def predict(self, sample):
        print(sample)
        likelihoods_list = [self.normal_dist(feature, self.likelihoods[i][0], self.likelihoods[i][1]) for i,feature in enumerate(sample)]
        print(likelihoods_list)



In [155]:
n = NaiveBayes()
n.build_classifier(x_train, y_train)
for i,sample in enumerate(x_test):
    y_prediction = n.predict(sample)
    if y_test[i] == y_prediction:
        print("Success")
    break

[6.3 2.5 4.9 1.5]
[2 0 2 2]
[0.2740740740740741, 0.15555555555555556, 0.4074074074074074, 0.28888888888888886]


In [156]:
g = GaussianNaiveBayes()
g.build_classifier(x_train, y_train)
for i,sample in enumerate(x_test):
    y_prediction = g.predict(sample)
    if y_test[i] == y_prediction:
        print("Success")
    break


[6.3 2.5 4.9 1.5]
[0.4166742044020813, 0.3941450497781225, 0.18151741396628798, 0.48368612309930614]
